# Capstone Project: ETL Pipeline for Fire Department Calls and Temperature Data

## Project Summary
The goal of this project: 
* Download San Francisco Fire department Calls-For-Service data set from Kaggle to local machine, 
* Create an ETL pipeline that extracts the data from .../input, processes data using Spark, and loads the data back into .../output as set of dimensional tables. <br>


This will allow analytics team to find insights of the fire calls-for-service.e.g.
* What were the common types of fire calls?<br> 
* Which months within the year 2019 saw for the highest number of fire calls? <br>
* Which neighborhood in SF generated the most fire calls in 2019?<br>
* What is the longest reaction time when a fire call is trigged till the service is available?<br>
* What is the fastest reaction time when a fire call is trigged till the service is available?<br>
* How to improve in order to reduce the fire incident?<br>


The project follows the follow steps:
* Step 1: Scope the Project and Gather Data<br>
* Step 2: Explore and Assess the Data<br>
* Step 3: Define the Data Model<br>
* Step 4: Run ETL to Model the Data<br>
* Step 5: Complete Project Write Up<br>

## Import Modules

In [62]:
# Do all imports and installs here
import pandas as pd
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

from pyspark.sql.types import *
import pyspark.sql.functions as F

import configparser
import os

## Create Spark Session

In [2]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark

spark = create_spark_session()

## Step 1: Scope the Project and Gather Data

### Scope
In this project, we will aggregate fire call data by CallType to form first dimension table. Next we will aggregate city temperature data by year and month to form the second dimension table. The two datasets will be joined on year and month to form the fact table. The final database is optimized to query on fire call events to determine if temperature affects the number of fire incidents. Spark will be used to process the data.

 
### Describe and Gather Data
**Data Source 1** <br>
San Francisco Fire Calls-For-Service <br>
https://www.kaggle.com/san-francisco/sf-fire-data-incidents-violations-and-more
    
**Content** <br>
Fire Calls-For-Service includes all fire units responses to calls. Each record includes the call number, incident number, address, unit identifier, call type, and disposition. All relevant time intervals are also included.

**Data Source 2**<br>

https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data?select=GlobalLandTemperaturesByCity.csv
The temperature data comes from Kaggle. It is provided in csv format. Some relevant attributes include:

**Content** <br>
AverageTemperature = average temperature
City = city name
Country = country name
Latitude= latitude
Longitude = longitude

### Read in fire calls data

In [64]:
# Read in the fire calls data 
input_path = "input/fire-department-calls-for-service.csv"
fire_df =spark.read.csv(input_path, header=True, mode="DROPMALFORMED")
fire_df.count()

5319351

In [65]:
fire_df.printSchema()

root
 |-- Call Number: string (nullable = true)
 |-- Unit ID: string (nullable = true)
 |-- Incident Number: string (nullable = true)
 |-- Call Type: string (nullable = true)
 |-- Call Date: string (nullable = true)
 |-- Watch Date: string (nullable = true)
 |-- Received DtTm: string (nullable = true)
 |-- Entry DtTm: string (nullable = true)
 |-- Dispatch DtTm: string (nullable = true)
 |-- Response DtTm: string (nullable = true)
 |-- On Scene DtTm: string (nullable = true)
 |-- Transport DtTm: string (nullable = true)
 |-- Hospital DtTm: string (nullable = true)
 |-- Call Final Disposition: string (nullable = true)
 |-- Available DtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode of Incident: string (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- Station Area: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- Original Priority: string (nullable = true)
 |-- Priority: string (nullable = 

### Read in temperature data

In [66]:
# Read in temperature data
temperature_input_path = "input/GlobalLandTemperaturesByCity.csv"
df_t =spark.read.csv(temperature_input_path, header=True)
df_t.count()

8599212

In [67]:
df_t.show(5, False)

+----------+------------------+-----------------------------+-----+-------+--------+---------+
|dt        |AverageTemperature|AverageTemperatureUncertainty|City |Country|Latitude|Longitude|
+----------+------------------+-----------------------------+-----+-------+--------+---------+
|1743-11-01|6.068             |1.7369999999999999           |Århus|Denmark|57.05N  |10.33E   |
|1743-12-01|null              |null                         |Århus|Denmark|57.05N  |10.33E   |
|1744-01-01|null              |null                         |Århus|Denmark|57.05N  |10.33E   |
|1744-02-01|null              |null                         |Århus|Denmark|57.05N  |10.33E   |
|1744-03-01|null              |null                         |Århus|Denmark|57.05N  |10.33E   |
+----------+------------------+-----------------------------+-----+-------+--------+---------+
only showing top 5 rows



## Step 2: Cleaning the data
For the temperature data,  drop all entries where AverageTemperature is NaN, then add year and month as new columns

In [9]:
# Filter out entries with NaN average temperature
df_t=df_t.filter(df_t.AverageTemperature != 'NaN')
df_t.show(5,False)

+----------+------------------+-----------------------------+-----+-------+--------+---------+
|dt        |AverageTemperature|AverageTemperatureUncertainty|City |Country|Latitude|Longitude|
+----------+------------------+-----------------------------+-----+-------+--------+---------+
|1743-11-01|6.068             |1.7369999999999999           |Århus|Denmark|57.05N  |10.33E   |
|1744-04-01|5.7879999999999985|3.6239999999999997           |Århus|Denmark|57.05N  |10.33E   |
|1744-05-01|10.644            |1.2830000000000001           |Århus|Denmark|57.05N  |10.33E   |
|1744-06-01|14.050999999999998|1.347                        |Århus|Denmark|57.05N  |10.33E   |
|1744-07-01|16.082            |1.396                        |Århus|Denmark|57.05N  |10.33E   |
+----------+------------------+-----------------------------+-----+-------+--------+---------+
only showing top 5 rows



In [10]:
df_t.count()

8235082

In [11]:
# Remove duplicate locations
df_t=df_t.dropDuplicates(['dt', 'City', 'Country'])
df_t.count()

8190783

In [12]:
# Add 2 new columns "year" and "month" to df_t
df_t = df_t.withColumn("year", F.year(df_t.dt)) \
                 .withColumn("month", F.month(df_t.dt))

In [13]:
df_t.show(2, False)

+----------+------------------+-----------------------------+-----------+-------+--------+---------+----+-----+
|dt        |AverageTemperature|AverageTemperatureUncertainty|City       |Country|Latitude|Longitude|year|month|
+----------+------------------+-----------------------------+-----------+-------+--------+---------+----+-----+
|1743-11-01|7.03              |1.611                        |Bielefeld  |Germany|52.24N  |7.88E    |1743|11   |
|1743-11-01|7.318             |1.764                        |Kaliningrad|Russia |55.45N  |19.84E   |1743|11   |
+----------+------------------+-----------------------------+-----------+-------+--------+---------+----+-----+
only showing top 2 rows



In [15]:
# Query temperature info for city "San Francisco"
df_t_san = (df_t.select('dt','year','month', 'Country', 'City', "AverageTemperature").where(df_t.City == "San Francisco")).sort('dt',ascending=False)

In [16]:
df_t_san.show(24)

+----------+----+-----+-------------+-------------+------------------+
|        dt|year|month|      Country|         City|AverageTemperature|
+----------+----+-----+-------------+-------------+------------------+
|2013-09-01|2013|    9|United States|San Francisco|            20.471|
|2013-08-01|2013|    8|United States|San Francisco|19.730999999999998|
|2013-07-01|2013|    7|United States|San Francisco|20.656999999999996|
|2013-06-01|2013|    6|United States|San Francisco|            19.759|
|2013-05-01|2013|    5|United States|San Francisco|            17.434|
|2013-04-01|2013|    4|United States|San Francisco|15.995999999999999|
|2013-03-01|2013|    3|United States|San Francisco|13.505999999999998|
|2013-02-01|2013|    2|United States|San Francisco|            10.229|
|2013-01-01|2013|    1|United States|San Francisco|              8.32|
|2012-12-01|2012|   12|United States|San Francisco|              8.95|
|2012-11-01|2012|   11|United States|San Francisco|            13.367|
|2012-

## Step 3: Define the Data Model


### 3.1 Conceptual Data Model
The first dimension table will contain events from the fire department calls data. The columns below will be extracted from the fire calls dataframe:

>Call Number<br>
>City<br>
>Call Type<br>
>Call year<br>
>call month<br>
>numberofcalls<br>
>Latitude<br>
>Longitude <br>

       
    
The second dimension table will contain city temperature data. The columns below will be extracted from the temperature dataframe:
>AverageTemperature <br>
>City <br>
>Country <br>
>Latitude= latitude<br>
>Longitude = longitude<br>

The fact table will contain information from the fire calls data joined with the city temperature data on City:

>Call Number<br>
>City<br>
>Call Type<br>
>Call year<br>
>call month<br>
>numberofcalls<br>
>Latitude<br>
>Longitude <br>
>AverageTemperature<br>
>Country<br>


The tables will be saved to Parquet files partitioned by city.

### 3.2 Mapping Out Data Pipelines
The pipeline steps are described below:
    

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

In [ ]:
#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

In [ ]:
#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.